<a href="https://colab.research.google.com/github/PapaZeusOne/Automated-Trading-Bot/blob/main/Automating_Stock_Trades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np 
import pandas as pd
import os, io
import datetime as dt
import pandas_datareader.data as web
import requests

# **TODO**


> Vergleiche mehrere Aktien miteinander



> Sende Push zu der Aktie, mit der besten Bewertung

# **Weitere Ideen**


> Aktien Sektoren zuordnen und den Bot nach Sektoren abfragen. Der Bot soll dann die stärkste Aktie aus dem Sektor zurückgeben. Ziemlich cool, wenn man weiß das zum Beispiel der Wasserstoffsektor gerade extrem am steigen ist.




> Sobald man sich für eine Aktie entschieden, wäre es ja auch interessant zu wissen, wann man am besten aussteigen sollte. Dann könnte man Abfragen schreiben und über den Bot eine Push rausschicken, ob man eine bestimmte Aktie, die man mit dem Bot teilt, verkaufen soll oder nicht, weil sich gerade der Kurs schlecht entwickelt.





# **Indikatoren**


> **MACD & Signallinie**: Der MACD (Moving Average Convergence/Divergence) wird in der Chartanalyse genutzt um Trends sowie Kauf-und Verkaufssignale zu erkennen. Dabei werden zwei exponentiell gewichtete gleitende Durchschnitte (26 und 12 Tage) berechnet und voneinander subtrahiert. Des Weiteren wird eine Signallinie berechnet. Hierbei handelt es sich um den 9 Tage exponentiell geglätteten Durchschnitt. Ein steigender MACD ist ein Indikator für einen Aufwärtstrend, ein sinkender MACD ein Indikator für einen Abwärtstrend. Schneidet der MACD die Signallinie von unten nach oben, so liegt ein Kaufsignal vor. Wird die Signallinie vom MACD von oben nach unten geschnitten, so liegt ein Verkaufssignal vor.



> **RSI**: Der RSI (Relative-Stärke-Index) ist ein Marktindikator. Er wird berechnet, indem der gleitende Mittelwert der Auf-und Abwärtsveränderungen für einen bestimmten Zeitraum (in der Regel 14 Tage) ermittelt wird. Der RSI kann Werte zwischen 0 und 100 einnehmen. Die Werte 30 und 70 spielen beim RSI eine besondere Rolle: Bei einem Wert unter 30 gilt der Wert als überverkauft. Dies bedeutet, dass eine Kurserholung (Steigerung) in nächster Zeit anstehen könnte. Bei einem Wert über 70 gilt die Aktie als überkauft. Dies ist ein Indiz dafür, dass der Kurs in nächster Zeit fallen könnte.




> **Bollinger Band**: Es existieren eigentlich zwei Bollinger Bänder, die gemeinsam mit dem gleitenden Mittel, dem arithmetischen Mittel der Kurse, dann das sogenannte Bollinger Band bilden. Mit Hilfe des Bollinger Bands werden der Verlauf der letzten Tage eines Wertpapieres und dessen Trend visualisiert. Man sieht, wo der Kurs durchschnittlich lag und wie stark er an welchen Tagen vom Mittelwert abwich. In der Regel verwendet man dafür den Zeitraum der letzten 20 Tage.





> **Bollinger Bands oder Keltner Channels?** Schau hier warum die BB die bessere Wahl wären: https://tradeciety.com/keltner-channel-vs-bollinger-bands-one-better/ Aber statt der Verwendung der Standardabweichung zur Darstellung der Bollinger Bänder wird die Average True Range von Wilders eingesetzt. Dieser Unterschied bewirkt, dass der Keltner Channel ruhiger und gleichmäßiger aussieht als die Bollinger Bänder.



> **EMA**: Exponential Moving Average verwendet man, um Kursschwankungen zu glätten und den zugrunde liegenden Trend dadurch besser zu erkennen. Hierzu betrachtet man einen Kursverlauf für einen bestimmten Zeitraum in der Vergangenheit und berechnet daraus den Durchschnitt. **Formel**: https://admiralmarkets.com/de/wissen/articles/forex-indicators/ema-der-exponentiell-gleitende-durchschnitt-als-universelles-hilfsmittel-fur-ihr-trading




> **Commodity Channel Index (CCI)**: Für den CCI wird die Abweichung zwischen Kurs und dessen Gleitenden Durchschnitt herangezogen, um das Momentum zu erfassen. Um dies zu bewerkstelligen, wird die Abweichung zwischen dem aktuellen Betrachtungszeitraum und der durchschnittlichen Abweichung der vergangenen X Zeitperioden verglichen. Im Gegensatz zum Stochastik-Oszillator und dem RSI sind die Werte des CCI nicht begrenzt. Aber in der Praxis bewegt sich der CCI aufgrund seiner Konstruktion zwischen -100 und +100. 
Bemerkenswerterweise werden beim CCI nicht die Schlusskurse, sondern die typischen Kurse (Hoch + Tief + Schlusskurs / 3) zugrunde gelegt. Dann wird die Abweichung bzw. Entfernung zwischen typischem Kurs und Gleitendem Durchschnitt ermittelt. **Formel**: https://www.kagels-trading.de/technische-indikatoren-trader/ **WICHTIG**: Wie der Name schon sagt, gründet der CCI auf der Annahme, dass sich die Kurse normalerweise innerhalb eines unsichtbaren Kanals um den Gleitenden Durchschnitt herum aufhalten.














# **Indikatoren mit denen wir anfangen sollten**


> 1. EMA


> 2. MACD


> 3. Bollinger Bänder



> 4. RSI












In [3]:
#indicator = pd.DataFrame(df, index=df.index)

In [4]:
def calc_indicators(df):
  ema_25 = 25
  ema_100 = 100
  ema_short = 12
  ema_long = 26
  bollinger_avg = 20
  signalline_span = 9

  df['ema_25'] = df['Close'].ewm(span=ema_25, adjust=False).mean()
  df['ema_100'] = df['Close'].ewm(span=ema_100, adjust=False).mean()

  df['short_avg_ma'] = df['Close'].rolling(window=bollinger_avg).mean()
  df['short_avg_std'] = df['Close'].rolling(window=bollinger_avg).std()

  df['bollinger'] = df['Close'].ewm(span=bollinger_avg, adjust=False).mean()
  df['bollinger_lower_band'] = df['short_avg_ma'] - (df['short_avg_std'] *2)
  df['bollinger_upper_band'] = df['short_avg_ma'] + (df['short_avg_std'] *2)

  df['ema_short'] = df['Close'].ewm(span=ema_short, adjust=False).mean()
  df['ema_long'] = df['Close'].ewm(span=ema_long, adjust=False).mean()
  df['macd'] = df['ema_short'] - df['ema_long']
  df['signalline'] = df['macd'].ewm(span=signalline_span, adjust=False).mean()

  df['RSI'] = calc_rsi(df)

  df = calculate_dead_cat_bounce_v2(df)

  return df

In [17]:
def calc_rsi(df, n=14):
    delta = df['Adj Close'].diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    # RolUp = pd.rolling_mean(dUp, n)
    # RolDown = pd.rolling_mean(dDown, n).abs()

    # Calculate the EWMA
    RolUp = dUp.ewm(span=n).mean()
    RolDown = dDown.abs().ewm(span=n).mean()

    # Calculate the RSI based on EWMA
    RS = RolUp / RolDown
    RSI = 100 - (100/(1+RS))

    # Calculate the SMA
    roll_up2 = dUp.rolling(n).mean()
    roll_down2 = dDown.abs().rolling(n).mean()

    # Calculate the RSI based on SMA
    RS2 = roll_up2 / roll_down2
    RSI2 = 100.0 - (100.0 / (1.0 + RS2))

    own_RSI=(RSI+RSI2)/2
    return own_RSI

In [6]:
def calculate_dead_cat_bounce_v2(df):
  set_freq = '2D'

  df['body_mid'] = (df['Close'] + df['Open'])/2

  pct_change_close = df['body_mid'].pct_change(freq=set_freq, fill_method='ffill')
  pct_change_close.dropna(inplace=True)

  dead_cat_bounce = np.where((df.signalline > df.macd) &
                            ((pct_change_close >= 0.05) | (pct_change_close <= -0.05)) &
                            ((df.Open < df.bollinger_lower_band)|(df.Close < df.bollinger_lower_band)),
                            True, False)

  df['dead_cat_bounce'] = dead_cat_bounce
  del df['body_mid']

  return df

In [7]:
def combine_stocks_to_dataframe(id_list):
  start = dt.datetime(2020,1,1)
  end = dt.datetime.now()
  df_list = []

  for id in id_list:
    stock = web.DataReader(id, 'yahoo', start, end) # load data
    stock['stock_id'] = id
    stock = calc_indicators(stock) # calc indicators for every stock
    stock.dropna(inplace=True) # maybe delete this later
    df_list.append(stock)

  df_complete = pd.concat(df_list)
  
  return df_complete

In [20]:
df_complete = combine_stocks_to_dataframe(['AMZN', 'V', 'AAPL', 'VOW.DE', 'PLUG'])
df_complete[df_complete.stock_id == 'V']

,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,short_avg_ma,short_avg_std,bollinger,bollinger_lower_band,bollinger_upper_band,ema_short,ema_long,macd,signalline,RSI,dead_cat_bounce
Date,,,,,,,,,,,,,,,,,,,,
2020-01-30,208.500000,203.320007,204.000000,208.210007,8571900.0,206.615479,V,199.422293,193.925422,198.934501,6.898715,200.580843,185.137071,212.731931,203.035396,199.220269,3.815127,3.275506,75.282178,False
2020-01-31,203.979996,198.259995,201.490005,198.970001,16699600.0,197.446213,V,199.387501,194.025314,199.327001,6.649522,200.427429,186.027957,212.626045,202.409951,199.201731,3.208220,3.262049,49.728397,False
2020-02-03,202.259995,199.850006,199.940002,200.809998,10887900.0,199.272141,V,199.496924,194.159664,199.887501,6.246719,200.463865,187.394063,212.380938,202.163804,199.320862,2.842942,3.178228,52.451339,False
2020-02-04,204.919998,202.229996,203.500000,203.559998,10205900.0,202.001068,V,199.809468,194.345810,200.606001,5.758903,200.758734,189.088195,212.123806,202.378603,199.634872,2.743731,3.091328,57.061389,False
2020-02-05,205.600006,199.949997,205.500000,202.809998,8672400.0,201.256805,V,200.040278,194.513417,201.312000,5.042097,200.954093,191.227805,211.396195,202.444972,199.870066,2.574905,2.988044,53.210431,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-19,227.580002,225.080002,225.750000,225.789993,6541600.0,225.789993,V,219.067971,212.021844,217.192499,6.328633,219.935254,204.535232,229.849766,221.846151,218.910765,2.935386,2.033447,74.237328,False
2021-04-20,225.380005,221.820007,224.779999,223.279999,5799100.0,223.279999,V,219.391973,212.244777,217.956499,6.077838,220.253802,205.800823,230.112174,222.066743,219.234412,2.832331,2.193223,67.098352,False
2021-04-21,227.869995,222.869995,223.309998,227.449997,9490600.0,227.449997,V,220.011821,212.545871,218.921499,5.970086,220.939154,206.981327,230.861671,222.894936,219.842974,3.051962,2.364971,75.119793,False


In [9]:
df_yd = df_complete.drop_duplicates(subset='stock_id', keep='last')

In [10]:
def compute_score(row):
  score=0
  if row['ema_25']>row['ema_100']:
    score+=1
    if row['macd']>row['signalline']:
      score+=1
      if (row['Close']<row['bollinger_lower_band']):
        score+=1
      if row['Close']>row['bollinger_upper_band']:
        score-=1
  return score

In [11]:
df_yd.apply(lambda row: compute_score(row), axis=1)

Date
2021-04-23    1
2021-04-23    2
2021-04-23    2
2021-04-23    1
2021-04-23    0
dtype: int64

In [12]:
df_complete[df_complete.dead_cat_bounce==True]

,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,short_avg_ma,short_avg_std,bollinger,bollinger_lower_band,bollinger_upper_band,ema_short,ema_long,macd,signalline,RSI,dead_cat_bounce
Date,,,,,,,,,,,,,,,,,,,,
2020-02-28,1889.760010,1811.130005,1814.630005,1883.750000,9493800.0,1883.750000,AMZN,2010.272724,1946.542323,2063.302502,89.953940,2015.396557,1883.394623,2243.210382,2008.919494,2008.991398,-0.071904,33.988335,27.849628,True
2020-03-12,1765.000000,1675.000000,1721.979980,1676.609985,11346200.0,1676.609985,AMZN,1931.792720,1933.753769,1972.158002,136.463919,1919.289229,1699.230163,2245.085840,1877.966015,1933.541686,-55.575671,-24.260448,26.428789,True
2020-02-27,186.339996,180.000000,182.119995,180.009995,17536600.0,178.890121,V,200.344327,196.500786,202.718999,8.695145,200.090730,185.328710,220.109288,198.012502,200.349486,-2.336985,0.991706,26.587887,True
2020-02-28,182.330002,172.979996,173.789993,181.759995,22674700.0,180.629242,V,198.914763,196.208890,201.396498,9.762047,198.344946,181.872404,220.920593,195.512116,198.972487,-3.460371,0.101290,29.316219,True
2020-03-12,172.000000,156.990005,158.500000,160.080002,27467700.0,159.084106,V,188.838081,193.653552,191.452998,15.242086,186.750095,160.968827,221.937170,181.313921,189.171830,-7.857909,-4.923505,28.519780,True
2020-10-28,184.699997,179.339996,183.979996,180.869995,17809500.0,180.317947,V,197.974008,197.419718,199.335999,5.977128,197.148028,187.381743,211.290256,194.841604,198.104320,-3.262716,-1.478273,17.302533,True
2020-02-27,71.500000,68.239998,70.275002,68.379997,320605600.0,67.854019,AAPL,77.370480,76.728716,78.426999,3.654900,77.100058,71.117199,85.736800,75.932890,77.402667,-1.469777,-0.116436,20.962643,True
2020-02-28,69.602501,64.092499,64.315002,68.339996,426884800.0,67.814331,AAPL,76.675828,76.562603,77.795624,4.237231,76.265767,69.321162,86.270086,74.764753,76.731358,-1.966606,-0.486470,22.030498,True
2020-03-12,67.500000,62.000000,63.985001,62.057499,418474000.0,61.580158,AAPL,73.290595,75.581426,73.744624,5.375677,72.489679,62.993269,84.495979,70.529630,73.422153,-2.892524,-1.966680,32.945711,True


In [13]:
def bot_warn_me(text):
    
    # Change the strings: <TOKEN> and <CHAT_ID>, with your token and chat_id
    bot_token_id = '1790740607:AAE7Omfuea3tG3iii9A1qX-vQJ9pdLFcO0s'
    bot_chat_id = '908941210'
    
    send_text = 'https://api.telegram.org/bot' + bot_token_id + '/sendMessage?chat_id=' + bot_chat_id + '&parse_mode=Markdown&text=' + text
    
    response = requests.get(send_text)

    return response.json()

In [14]:
dead_cat_bounce = calculate_dead_cat_bounce()
#bot_warn_me('Watch Out! Check Code')
dead_cat_bounce

NameError: name 'calculate_dead_cat_bounce' is not defined